<a href="https://colab.research.google.com/github/SatyanarayanaMurthyT/AIML-Projects/blob/ECG-Digitization/ECGDigitization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wfdb
!pip install qrcode
!pip install validators

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00


In [2]:
# prompt: import pandas

import pandas as pd
import IPython
import zipfile
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
!lscpu

print(".......................................")
!lscpu |grep 'Model name'
print(".......................................")
!free -h --si | awk  '/Mem:/{print $2}'



Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4400.32
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
!pwd
!ls /content/drive/MyDrive/Colab\ Notebooks/ECGDigitization/

/content
ECGDigitization.ipynb  Image_Files	      ptb-xl.zip	     st12slv23ToSNOMED.csv
ecg-image-generator    prepare_image_data.py  remove_hidden_data.py
helper_code.py	       prepare_ptbxl_data.py  st12sl_statements.csv


In [6]:
ECGDir="/content/drive/MyDrive/Colab Notebooks/ECGDigitization/Image_Files/ECGImageAnalysis.zip"
ECGDir

'/content/drive/MyDrive/Colab Notebooks/ECGDigitization/ptb-xl.zip'

In [7]:
with zipfile.ZipFile(ECGDir,"r") as zip_ref:
    zip_ref.extractall("./")

In [8]:
!mv "/content/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3" "/content/ECGImageAnalysis"

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/ECGDigitization/prepare_ptbxl_data.py" "./ECGImageAnalysis"
!cp "/content/drive/MyDrive/Colab Notebooks/ECGDigitization/st12slv23ToSNOMED.csv" "./ECGImageAnalysis"
!cp "/content/drive/MyDrive/Colab Notebooks/ECGDigitization/st12sl_statements.csv" "./ECGImageAnalysis"
!cp "/content/drive/MyDrive/Colab Notebooks/ECGDigitization/helper_code.py" "./ECGImageAnalysis"
!cp "/content/drive/MyDrive/Colab Notebooks/ECGDigitization/prepare_image_data.py" "./ECGImageAnalysis"
!cp "/content/drive/MyDrive/Colab Notebooks/ECGDigitization/remove_hidden_data.py" "./ECGImageAnalysis"


In [ ]:
!pwd
!ls

/content
drive  ptb-xl  sample_data


In [ ]:
!ls ptb-xl/

example_physionet.py   prepare_ptbxl_data.py	 RECORDS		scp_statements.csv
helper_code.py	       ptbxl_database.csv	 records100		SHA256SUMS.txt
LICENSE.txt	       ptbxl_v102_changelog.txt  records500		st12sl_statements.csv
prepare_image_data.py  ptbxl_v103_changelog.txt  remove_hidden_data.py	st12slv23ToSNOMED.csv


In [ ]:
!python  ptb-xl/prepare_ptbxl_data.py \
     -i  ptb-xl/records500/ \
     -pd ptb-xl/ptbxl_database.csv \
     -pm ptb-xl/scp_statements.csv \
     -sd ptb-xl/st12sl_statements.csv \
     -sm ptb-xl/st12slv23ToSNOMED.csv \
     -o  ptb-xl/records500_tmp/

In [ ]:
import os
print(os.path.exists("/content/ptb-xl/ecg_image"))
print(os.path.isdir("/content/ptb-xl/ecg_image/ptb-xl/records500/00000"))


False
False


In [ ]:

import shutil

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.exists(d):
            try:
                shutil.rmtree(d)
            except Exception as e:
                print(e)
                os.unlink(d)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)
copytree("/content/drive/MyDrive/Colab Notebooks/ECGDigitization/ecg-image-generator/", "./ptb-xl/" )

In [ ]:
import subprocess
import os
import shutil
import threading

In [ ]:
def generateECGImages(image_dir):
 tmp_dir="records500_tmp/"+image_dir
 print("image dir: ",tmp_dir)
 arg1= " -i "+tmp_dir
 arg2= " -o "+tmp_dir
 arg3=" --print_header"
 arg4=" --store_config 2"

 str_arg_cmd="python ptb-xl/gen_ecg_images_from_data_batch.py"+arg1+arg2+arg3+arg4
 print(str_arg_cmd)
 os.system(str_arg_cmd)

def generateECGImagesForDir(image_dir):
  generateECGImages(image_dir)
  shutil.make_archive("ptb-xl/records500_tmp/"+image_dir, 'zip', "ptb-xl/records500_tmp/"+image_dir)
  input_zip="ptb-xl/records500_tmp/"+image_dir+".zip"
  output_zip="/content/drive/MyDrive/Colab Notebooks/ECGDigitization/Image_Files/"+image_dir+".zip"
  shutil.move(input_zip,output_zip)

list_dir=sorted(os.listdir("ptb-xl/records500_tmp/"))
for image_dir in list_dir[18:]:
  print("image directory: ",image_dir)
  generateECGImagesForDir(image_dir)


image directory:  18000
image dir:  records500_tmp/18000
python ptb-xl/gen_ecg_images_from_data_batch.py -i records500_tmp/18000 -o records500_tmp/18000 --print_header --store_config 2
image directory:  19000
image dir:  records500_tmp/19000
python ptb-xl/gen_ecg_images_from_data_batch.py -i records500_tmp/19000 -o records500_tmp/19000 --print_header --store_config 2
image directory:  20000
image dir:  records500_tmp/20000
python ptb-xl/gen_ecg_images_from_data_batch.py -i records500_tmp/20000 -o records500_tmp/20000 --print_header --store_config 2
image directory:  21000
image dir:  records500_tmp/21000
python ptb-xl/gen_ecg_images_from_data_batch.py -i records500_tmp/21000 -o records500_tmp/21000 --print_header --store_config 2
